In [1]:
# -*- coding: utf-8 -*-
"""
对 6 个插值后的文件：
1. 截取 1850-1949 (1200 months)
2. 仅提取主要变量 (gpp 或 tos)
3. 定义索引数组 XXXX = 1..1200
4. 沿 time 方向做线性去趋势
5. 保存结果 (_1850-1949_detrend.nc)
"""

import xarray as xr
import numpy as np
from pathlib import Path

DATA_DIR = Path(r"D:\cangku\data")

FILES = [
    "gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc",
    "gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc",
    "gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc",
    "tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc",
    "tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc",
    "tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc",
]

# 1D 索引数组 1..1200
XXXX = np.arange(1, 1201, dtype="float64")

def detrend_3d(data):
    """
    输入: data (time, lat, lon) -> numpy array
    输出: 去趋势后的 data (time, lat, lon)
    """
    t, nlat, nlon = data.shape
    out = np.full_like(data, np.nan, dtype="float64")

    for i in range(nlat):
        for j in range(nlon):
            y = data[:, i, j]
            if np.all(np.isnan(y)):
                continue
            mask = np.isfinite(y)
            if mask.sum() > 1:
                p = np.polyfit(XXXX[mask], y[mask], 1)
                trend = np.polyval(p, XXXX)
                out[:, i, j] = y - trend
    return out

def process_one(path: Path):
    with xr.open_dataset(path) as ds:
        # 识别主要变量 gpp/tos
        main_var = None
        for cand in ["gpp", "tos"]:
            if cand in ds.data_vars:
                main_var = cand
                break
        if main_var is None:
            print(f"[跳过] {path.name}: 没找到 gpp 或 tos")
            return

        # 裁剪时间 1850-1949
        da = ds[main_var].sel(time=slice("1850-01-01", "1949-12-31")).astype("float64")

        # 转 numpy (time, lat, lon)
        data = da.values
        if da.dims != ("time", "lat", "lon"):
            raise ValueError(f"{main_var} 维度顺序异常: {da.dims}")

        # 去趋势
        detrended = detrend_3d(data)

        # 转回 DataArray
        da_dt = xr.DataArray(
            detrended,
            coords={"time": da["time"], "lat": da["lat"], "lon": da["lon"]},
            dims=("time", "lat", "lon"),
            name=main_var,
            attrs=da.attrs,
        )

        # 存储
        out_path = path.with_name(path.stem + "_1850-1949_detrend.nc")
        da_dt.to_netcdf(out_path)
        print(f"[完成] {path.name} -> {out_path.name}")

def main():
    for fname in FILES:
        path = DATA_DIR / fname
        if path.exists():
            try:
                process_one(path)
            except Exception as e:
                print(f"[错误] {fname}: {e}")
        else:
            print(f"[缺失] {fname}")

if __name__ == "__main__":
    main()


[完成] gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc -> gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1_1850-1949_detrend.nc
[完成] gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc -> gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1850-1949_detrend.nc
[完成] gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc -> gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_1850-1949_detrend.nc
[完成] tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc -> tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1_1850-1949_detrend.nc
[完成] tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc -> tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1850-1949_detrend.nc
[完成] tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc -> tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_1850-1949_detrend.nc
